In [1]:
from matplotlib import pyplot as plt

import numpy as np
np.random.seed(42)
import random
random.seed(42)

In [2]:
from matplotlib import pyplot as plt
import numpy as np
#plt.hist(train_dataset.target,bins=20, alpha=0.5, histtype='bar', ec='black',
#        label=categories)

In [3]:
import nltk
from nltk.corpus import stopwords
#nltk.download('wordnet')
#nltk.download('stopwords')
# nltk.download('punkt')#, if you need "tokenizers/punkt/english.pickle", choose it
# nltk.download('averaged_perceptron_tagger')


In [4]:
from nltk import pos_tag
wnl = nltk.wordnet.WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'

def lemmatize_sent(list_word):
    # Text input is string, returns array of lowercased strings(words).
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

def stem_lemma(doc):
    return (word for word in lemmatize_sent(analyzer(doc)) if not word.isdigit())

def stem_nolemma(doc):
    return (word for word in analyzer(doc) if not word.isdigit())

In [6]:
vectorizer_df3_lemma = CountVectorizer(min_df=3, 
                             analyzer=stem_lemma, 
                             stop_words='english')
vectorizer_df5_lemma = CountVectorizer(min_df=5, 
                             analyzer=stem_lemma, 
                             stop_words='english')
vectorizer_df3_nolemma = CountVectorizer(min_df=3, 
                             analyzer=stem_nolemma, 
                             stop_words='english')
vectorizer_df5_nolemma = CountVectorizer(min_df=5, 
                             analyzer=stem_nolemma, 
                             stop_words='english')


In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.base import BaseEstimator, TransformerMixin
class SparseToDenseArray(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def transform(self, X, *_):
        if hasattr(X, 'toarray'):
            return X.toarray()
        return X

    def fit(self, *_):
        return self

In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

## Question 8

### Part 1: Keep "headers" and "footers"

#### Classify the documents into two classes

In [9]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'comp.os.ms-windows.misc',
'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
'rec.autos', 'rec.motorcycles',
'rec.sport.baseball', 'rec.sport.hockey']
train_dataset = fetch_20newsgroups(subset = 'train', categories = categories
, shuffle = True, random_state = 42)
test_dataset = fetch_20newsgroups(subset = 'test', categories = categories
, shuffle = True, random_state = 42)

In [10]:
_, idx = np.unique(train_dataset.target, return_index=True)
for i in range(len(idx)):
    j = np.sort(idx)[i]
    #print(train_dataset.target_names[i],': target index = ', train_dataset.target[j])

#print(train_dataset.target[15:25])
#print(train_dataset.target_names[15:25])

In [11]:
train_dataset.bintarget = np.zeros_like(train_dataset.target)
test_dataset.bintarget = np.zeros_like(test_dataset.target)
for y in [6, 7, 4, 2]:
    train_dataset.bintarget[np.argwhere(train_dataset.target == y)] = 0
    test_dataset.bintarget[np.argwhere(test_dataset.target == y)] = 0
for y in [1, 3, 0, 5]:
    train_dataset.bintarget[np.argwhere(train_dataset.target == y)] = 1
    test_dataset.bintarget[np.argwhere(test_dataset.target == y)] = 1
class_names = ['Computer Technology', 'Recreational Activity']
#print(train_dataset.bintarget[:20])

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

# used to cache results
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.externals.joblib import Memory
# print(__doc__)
location = mkdtemp()
memory = Memory(location=location, verbose=10)

pipeline1 = Pipeline([
    ('vect', vectorizer_df3_lemma),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(n_components=50, random_state=42)),
    ('toarr', SparseToDenseArray()),
    ('clf', SVC(kernel='linear', C=33.0)),
],
memory=memory
)

param_grid1 = [
    {
        'vect': [vectorizer_df3_lemma, 
                 vectorizer_df3_nolemma, 
                 vectorizer_df5_lemma, 
                 vectorizer_df5_nolemma],
        'reduce_dim': [TruncatedSVD(n_components=50, random_state=42),
                       NMF(n_components=50, init='random', random_state=42)],
        'clf': [SVC(kernel='linear', C=33.0), 
                LogisticRegression(penalty='l1', C=3.3), 
                LogisticRegression(penalty='l2', C=3.3),
                GaussianNB()]
    }
]
grid1 = GridSearchCV(pipeline1, cv=5, n_jobs=1, param_grid=param_grid1, scoring='accuracy')
grid1.fit(train_dataset.data, train_dataset.bintarget)
rmtree(location)

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(CountVectorizer(analyzer=<function stem_lemma at 0x7fc88283f8c8>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=3, ngram_range=(1, 1), preprocessor=None,
        stop_words='english', strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None), 
[ 'From: wijkstra@fwi.uva.nl (Marcel Wijkstra (AIO))\n'
  'Subject: Re: BW hardcopy of colored window?\n'
  'Keywords: color hardcopy print\n'
  'Nntp-Posting-Host: ic.fwi.uva.nl\n'
  'Organization: FWI, University of Amsterdam\n'
  'Lines: 38\n'
  '\n'
  'mars@ixos.de (Martin Stein) writes:\n'
  '\n'
  '#I use xwd/xpr (from the X11R5 dist.) and various programs of the\n'
  '#ppm-tools to print hardcopies of colored X windows

/home/zeyudesk/anaconda3/envs/219project1/lib/python3.7/site-packages/sklearn/pipeline.py:230: UserWarning: Persisting input arguments took 1.36s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])


_______________________________________________fit_transform_one - 43.5s, 0.7min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), <3784x14478 sparse matrix of type '<class 'numpy.int64'>'
	with 431577 stored elements in Compressed Sparse Row format>, 
array([1, ..., 0]), None)
________________________________________________fit_transform_one - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=42, tol=0.0), 
<3784x14478 sparse matrix of type '<class 'numpy.float64'>'
	with 431577 stored elements in Compressed Sparse Row format>, 
array([1, ..., 0]), None)
________________________________________________f

/home/zeyudesk/anaconda3/envs/219project1/lib/python3.7/site-packages/sklearn/pipeline.py:230: UserWarning: Persisting input arguments took 1.41s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])


_______________________________________________fit_transform_one - 41.4s, 0.7min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), <3786x14065 sparse matrix of type '<class 'numpy.int64'>'
	with 423947 stored elements in Compressed Sparse Row format>, 
array([0, ..., 0]), None)
________________________________________________fit_transform_one - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=42, tol=0.0), 
<3786x14065 sparse matrix of type '<class 'numpy.float64'>'
	with 423947 stored elements in Compressed Sparse Row format>, 
array([0, ..., 0]), None)
________________________________________________f

...

[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/d649361cc13d7a5bb093722c929d8d4a
___________________________________fit_transform_one cache loaded - 0.2s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/815440b3ccf515db2483ced214cbf930
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/2bc7d5037667b172f99faed7b51cf309
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/99a82f69c2da9783d9b610cd48c77497
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmw

[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/ab12d8f7ac738216f610f2e8eefc57df
___________________________________fit_transform_one cache loaded - 0.3s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/9ca1e66895f908584d3ebd04d2217c8c
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/8dc9fe3f8b54e799fb3de75b025b6bf6
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/92308f694ca3e23a68a9d87df7074bd3
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmw

[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/db17f67e147d6624871d2770c3a399a7
___________________________________fit_transform_one cache loaded - 0.2s, 0.0min
[Memory]0.2s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/aa54b68679ebd02ae12061128afe15b8
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/ed3dd16452b7cc6a5a08c5b0a9c983f8
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmwtc/joblib/sklearn/pipeline/_fit_transform_one/ef4ee035f4621ae779555121b86f11a5
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmpp0ywmw

/home/zeyudesk/anaconda3/envs/219project1/lib/python3.7/site-packages/sklearn/pipeline.py:230: UserWarning: Persisting input arguments took 1.52s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])


_______________________________________________fit_transform_one - 48.9s, 0.8min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), <4732x16600 sparse matrix of type '<class 'numpy.int64'>'
	with 541291 stored elements in Compressed Sparse Row format>, 
array([0, ..., 0]), None)
________________________________________________fit_transform_one - 0.1s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=42, tol=0.0), 
<4732x16600 sparse matrix of type '<class 'numpy.float64'>'
	with 541291 stored elements in Compressed Sparse Row format>, 
array([0, ..., 0]), None)
________________________________________________f

In [13]:
pd.DataFrame(grid1.cv_results_)

/home/zeyudesk/anaconda3/envs/219project1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/zeyudesk/anaconda3/envs/219project1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/zeyudesk/anaconda3/envs/219project1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_reduce_dim,param_vect,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,44.962940,1.599756,10.275888,0.818540,"SVC(C=33.0, cache_size=200, class_weight=None,...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.886076,0.883721,...,0.878276,0.006130,1,0.890063,0.882726,0.882726,0.881933,0.890914,0.885672,0.003952
1,4.133561,0.082082,0.235809,0.009318,"SVC(C=33.0, cache_size=200, class_weight=None,...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.873418,0.868922,...,0.871090,0.002799,11,0.887685,0.869783,0.872689,0.873745,0.877707,0.876322,0.006223
2,44.577491,1.414655,10.860715,0.597864,"SVC(C=33.0, cache_size=200, class_weight=None,...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.887131,0.885835,...,0.876796,0.008531,3,0.889799,0.880349,0.880613,0.881933,0.889593,0.884457,0.004311
3,3.702933,0.082892,0.245684,0.010211,"SVC(C=33.0, cache_size=200, class_weight=None,...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.866034,0.868922,...,0.870456,0.003503,12,0.881871,0.874274,0.874010,0.879028,0.880085,0.877853,0.003165
4,17.387508,2.127713,10.570978,0.506719,"SVC(C=33.0, cache_size=200, class_weight=None,...","NMF(alpha=0.0, beta_loss='frobenius', init='ra...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.862869,0.873150,...,0.872992,0.009659,10,0.873943,0.876387,0.881141,0.875594,0.873745,0.876162,0.002681
5,18.279862,1.143657,0.321857,0.007318,"SVC(C=33.0, cache_size=200, class_weight=None,...","NMF(alpha=0.0, beta_loss='frobenius', init='ra...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.844937,0.829810,...,0.854818,0.016519,19,0.856765,0.830428,0.863708,0.866086,0.861595,0.855717,0.013012
6,11.254425,1.557047,9.709062,0.441352,"SVC(C=33.0, cache_size=200, class_weight=None,...","NMF(alpha=0.0, beta_loss='frobenius', init='ra...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.873418,0.865751,...,0.875106,0.007291,7,0.879228,0.877707,0.884575,0.870840,0.884839,0.879438,0.005148
7,13.374385,0.243960,0.300103,0.008307,"SVC(C=33.0, cache_size=200, class_weight=None,...","NMF(alpha=0.0, beta_loss='frobenius', init='ra...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.847046,0.837209,...,0.851014,0.016329,22,0.859408,0.845219,0.846012,0.861859,0.873217,0.857143,0.010506
8,0.372102,0.037255,9.660092,0.551356,"LogisticRegression(C=3.3, class_weight=None, d...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': LogisticRegression(C=3.3, class_weight...",0.881857,0.882664,...,0.875740,0.005347,5,0.887421,0.878236,0.881141,0.881933,0.889857,0.883718,0.004273
9,0.369887,0.015387,0.180031,0.006401,"LogisticRegression(C=3.3, class_weight=None, d...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': LogisticRegression(C=3.3, class_weight...",0.867089,0.867865,...,0.868766,0.004431,13,0.885307,0.869519,0.873481,0.870312,0.875330,0.874790,0.005665


### Model 0 has the best performance according to 5-fold cross validation, which has a mean validation score of 0.878276. The best combination of parameters are:
#### min_df = 3
#### lemmatization: on
#### classifier: SVM with $\gamma\$ = 33.0

In [22]:
# save the model and cross validation results
import pickle
# save the model to disk using python built-in method
filename = 'grid1.sav'
pickle.dump(grid1, open(filename, 'wb'))

In [23]:
# load the model and show results
loaded_model = pickle.load(open('grid1.sav', 'rb'))
#pd.DataFrame(loaded_model.cv_results_)

In [24]:
from sklearn.externals import joblib
# save the model to disk using scikit-learn package
filename = 'grid1_skl.sav'
joblib.dump(grid1, filename)

['grid1_skl.sav']

In [25]:
# load the model and show results
loaded_model = joblib.load('grid1_skl.sav')
#pd.DataFrame(loaded_model.cv_results_)

### Part 2: Remove "headers" and "footers"

#### Classify the documents into two classes

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'comp.os.ms-windows.misc',
'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
'rec.autos', 'rec.motorcycles',
'rec.sport.baseball', 'rec.sport.hockey']
train_dataset = fetch_20newsgroups(subset = 'train', categories = categories
, shuffle = True, random_state = 42, remove = ('headers', 'footers'))
test_dataset = fetch_20newsgroups(subset = 'test', categories = categories
, shuffle = True, random_state = 42, remove = ('headers', 'footers'))

In [ ]:
_, idx = np.unique(train_dataset.target, return_index=True)
for i in range(len(idx)):
    j = np.sort(idx)[i]
    #print(train_dataset.target_names[i],': target index = ', train_dataset.target[j])

#print(train_dataset.target[15:25])
#print(train_dataset.target_names[15:25])

In [ ]:
train_dataset.bintarget = np.zeros_like(train_dataset.target)
test_dataset.bintarget = np.zeros_like(test_dataset.target)
for y in [6, 7, 4, 2]:
    train_dataset.bintarget[np.argwhere(train_dataset.target == y)] = 0
    test_dataset.bintarget[np.argwhere(test_dataset.target == y)] = 0
for y in [1, 3, 0, 5]:
    train_dataset.bintarget[np.argwhere(train_dataset.target == y)] = 1
    test_dataset.bintarget[np.argwhere(test_dataset.target == y)] = 1
class_names = ['Computer Technology', 'Recreational Activity']
#print(train_dataset.bintarget[:20])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

# used to cache results
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.externals.joblib import Memory
# print(__doc__)
location = mkdtemp()
memory = Memory(location=location, verbose=10)

pipeline2 = Pipeline([
    ('vect', vectorizer_df3_lemma),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(n_components=50, random_state=42)),
    ('toarr', SparseToDenseArray()),
    ('clf', SVC(kernel='linear', C=33.0)),
],
memory=memory
)

param_grid2 = [
    {
        'vect': [vectorizer_df3_lemma, 
                 vectorizer_df3_nolemma, 
                 vectorizer_df5_lemma, 
                 vectorizer_df5_nolemma],
        'reduce_dim': [TruncatedSVD(n_components=50, random_state=42),
                       NMF(n_components=50, init='random', random_state=42)],
        'clf': [SVC(kernel='linear', C=33.0), 
                LogisticRegression(penalty='l1', C=3.3), 
                LogisticRegression(penalty='l2', C=3.3),
                GaussianNB()]
    }
]
grid2 = GridSearchCV(pipeline2, cv=5, n_jobs=1, param_grid=param_grid2, scoring='accuracy')
grid2.fit(train_dataset.data, train_dataset.bintarget)
rmtree(location)

In [ ]:
pd.DataFrame(grid2.cv_results_)